In [1]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("YoungWon")

LangSmith 추적을 시작합니다.
[프로젝트명]
YoungWon


### RunnablePassthrough

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# api 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# LLM 모델 생성
model = ChatOpenAI(
    api_key = openai_api_key,
    model = "gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0
)

# 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template("{num}의 10배는?")

# 체인 생성
chain = prompt | model

In [3]:
chain.invoke({"num":5})

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e4413697-514c-4b6e-9382-4dbd3022a18d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

In [4]:
# 1개의 변수만 템플릿에 포함하고 있다면 값만 전달하는 것도 가능함
chain.invoke(5)

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-210ebe73-ef8c-4e50-aaf0-daff89e6b512-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

In [5]:
# RunnablePassthrough를 사용한 예제
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num":10})

{'num': 10}

In [6]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | model

# dict 값이 RunnablePassthrough로 변경되었음
runnable_chain.invoke(10)

AIMessage(content='100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 15, 'total_tokens': 18, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-87b1c5ae-5697-4f7a-8d85-ca5186fed3d3-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18})

In [7]:
RunnablePassthrough().invoke({"num":1})

{'num': 1}

In [8]:
# 입력 키 : num, 할당(assign)키 : new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num":1})

{'num': 1, 'new_num': 3}

### RunnableParallel

In [9]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성
runnable = RunnableParallel(
    # RunnableParallel 인스턴스를 'passed'키워드 인자로 전달 (입력된 데이터를 그대로 통과시키는 역할)
    passed=RunnablePassthrough(),
    
    # 'extra' 키워드 인자로 'mult'람다 함수를 할당함 (num 값을 3배로 증가시킨다)
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    
    # 'modified' 키워드 인자로 람다 함수를 전달함. (num 값에 1을 더함)
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출한다. 
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [10]:
chain1 = (
    {"country":RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는?")
    | model
)

chain2 = (
    {"country":RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 인구수는?")
    | model
)

combined_chain = RunnableParallel(capital=chain1, number=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 18, 'total_tokens': 23, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-27fdfc08-8b3e-46a3-bef1-f415c58caba0-0', usage_metadata={'input_tokens': 18, 'output_tokens': 5, 'total_tokens': 23}),
 'number': AIMessage(content='2021년 9월 기준으로 대한민국의 인구수는 약 5천만 명입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 19, 'total_tokens': 53, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3e1cda39-7958-420e-8d54-a8d450e26c1b-0', usage_metadata={'input_tokens': 19, 'output_tokens': 34, 'total_tokens': 53})}

### RunnableLambda

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    return datetime.today().strftime("%b-%d")

get_today(None)

'Sep-20'

In [15]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요"
)

llm = ChatOpenAI(
    api_key = openai_api_key,
    model = "gpt-4-turbo",
    temperature = 0
)

# chain 생성
chain = (
    {"today": RunnableLambda(get_today), 
     "n":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke(3))

다음은 9월 20일에 태어난 유명인 세 명과 그들의 생년월일입니다:

1. 소피아 로렌 (Sophia Loren) - 1934년 9월 20일
   이탈리아 출신의 유명한 배우로, 그녀는 여러 국제적인 상을 수상하며 세계적으로 인정받은 연기자입니다.

2. 조지 R. R. 마틴 (George R. R. Martin) - 1948년 9월 20일
   미국의 판타지, 호러, 과학 소설 작가로 가장 잘 알려진 작품은 "얼음과 불의 노래" 시리즈로, 이는 "왕좌의 게임" TV 시리즈의 원작입니다.

3. 문근영 (Moon Geun-young) - 1987년 9월 20일
   대한민국의 배우로, 어린 시절부터 연기를 시작하여 "괴물", "청소년의 덫", "가을동화" 등 다양한 드라마와 영화에서 활약하였습니다.
